In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
import sys
sys.path.append("/n/home12/binxuwang/Github/mini_edm")
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning")
sys.path.append("/n/home12/binxuwang/Github/DiT")

In [6]:
import os
from os.path import join
import pickle as pkl
import torch
from tqdm import tqdm
import numpy as np
import seaborn as sns
import einops
import matplotlib.pyplot as plt
from easydict import EasyDict as edict
import torch as th

In [4]:
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.dpi'] = 72
plt.rcParams['figure.figsize'] = [6.0, 4.0]
plt.rcParams['figure.edgecolor'] = (1, 1, 1, 0)
plt.rcParams['figure.facecolor'] = (1, 1, 1, 0)
# vector graphics type
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [5]:
from train_edm import create_model, edm_sampler, EDM
from edm_utils import edm_sampler_inpaint, create_edm, get_default_config
from rule_utils import get_rule_img, get_obj_list, get_rule_list
from rule_utils import check_consistent
from dataset_utils import train_data2attr_tsr,load_raw_data,load_PGM_abstract

### Load in model

In [15]:
DiT_configs = {
    # "DiT_XL_2": {"depth": 28, "hidden_size": 1152, "patch_size": 2, "num_heads": 16},
    "DiT_XL_1": {"depth": 28, "hidden_size": 1152, "patch_size": 1, "num_heads": 16},
    "DiT_XL_3": {"depth": 28, "hidden_size": 1152, "patch_size": 3, "num_heads": 16},
    # "DiT_XL_4": {"depth": 28, "hidden_size": 1152, "patch_size": 4, "num_heads": 16},
    # "DiT_XL_8": {"depth": 28, "hidden_size": 1152, "patch_size": 8, "num_heads": 16},
    # "DiT_L_2": {"depth": 24, "hidden_size": 1024, "patch_size": 2, "num_heads": 16},
    "DiT_L_1": {"depth": 24, "hidden_size": 1024, "patch_size": 1, "num_heads": 16},
    "DiT_L_3": {"depth": 24, "hidden_size": 1024, "patch_size": 3, "num_heads": 16},
    # "DiT_L_4": {"depth": 24, "hidden_size": 1024, "patch_size": 4, "num_heads": 16},
    # "DiT_L_8": {"depth": 24, "hidden_size": 1024, "patch_size": 8, "num_heads": 16},
    # "DiT_B_2": {"depth": 12, "hidden_size": 768, "patch_size": 2, "num_heads": 12},
    "DiT_B_1": {"depth": 12, "hidden_size": 768, "patch_size": 1, "num_heads": 12},
    "DiT_B_3": {"depth": 12, "hidden_size": 768, "patch_size": 3, "num_heads": 12},
    # "DiT_B_4": {"depth": 12, "hidden_size": 768, "patch_size": 4, "num_heads": 12},
    # "DiT_B_8": {"depth": 12, "hidden_size": 768, "patch_size": 8, "num_heads": 12},
    # "DiT_S_2": {"depth": 12, "hidden_size": 384, "patch_size": 2, "num_heads": 6},
    "DiT_S_1": {"depth": 12, "hidden_size": 384, "patch_size": 1, "num_heads": 6},
    "DiT_S_3": {"depth": 12, "hidden_size": 384, "patch_size": 3, "num_heads": 6},
    # "DiT_S_4": {"depth": 12, "hidden_size": 384, "patch_size": 4, "num_heads": 6},
    # "DiT_S_8": {"depth": 12, "hidden_size": 384, "patch_size": 8, "num_heads": 6},
}

In [16]:
def infer_config_from_expname():
    pass

In [175]:
from diffusion import create_diffusion
from models import DiT

device = "cuda"
diffusion_eval = create_diffusion(timestep_respacing="ddim200")  # default: ddim100

In [17]:
class_dropout_prob = 1.0
num_classes = 0
model_cfg = DiT_configs["DiT_S_1"]
model = DiT(input_size=9,
            in_channels=3, **model_cfg,
            mlp_ratio=4.0,
            class_dropout_prob=class_dropout_prob,
            num_classes=num_classes,
            learn_sigma=True,)

In [18]:
exproot = r"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results"
expname = r"045-RAVEN10_abstract-uncond-DiT_S_1_20240311-1256"
expdir = join(exproot, expname)
ckptdir = join(expdir, "checkpoints")

ckpt_path = join(ckptdir, "1000000.pt")
state_dict = th.load(ckpt_path, )

In [23]:
model.load_state_dict(state_dict["model"])
model.to("cuda").eval();

In [27]:
# abstract RAVEN dataset
dataset_Xmean = th.tensor([1.5, 2.5, 2.5]).view(1, 3, 1, 1).to("cuda")
dataset_Xstd = th.tensor([2.5, 3.5, 3.5]).view(1, 3, 1, 1).to("cuda")


In [32]:
y = th.zeros(100, dtype=torch.int, device="cuda")
model_kwargs = dict(y=y)
with th.no_grad():
    samples = diffusion_eval.ddim_sample_loop(model, shape=(100, 3, 9, 9), clip_denoised=False, device="cuda", model_kwargs=model_kwargs, progress=True)
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()

  0%|          | 0/100 [00:00<?, ?it/s]

In [33]:
(samples - torch.round(samples)).abs().mean()

tensor(0.0075)

In [48]:
from rule_new_utils import check_r3_r2_batch

def infer_rule_from_sample_batch(sample_batch):
    # if not int convert to int
    sample_batch = sample_batch.round().int()
    sample_batch = sample_batch.view(-1, 3, 3, 3, 9) 
    sample_batch = einops.rearrange(sample_batch, 
        "B attr row h (panel w) -> B row panel (h w) attr", 
        panel=3, w=3, h=3, attr=3)
    r3_list, r2_list, rule_col = check_r3_r2_batch(sample_batch)
    return r3_list, r2_list, rule_col

In [39]:

sample_batch = samples.round().int().view(-1, 3, 3, 3, 9) 
sample_batch = einops.rearrange(sample_batch, 
        "B attr row h (panel w) -> B row panel (h w) attr", 
        panel=3, w=3, h=3, attr=3)
r3_list, r2_list, rule_col = check_r3_r2_batch(sample_batch)

In [49]:
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)

### Inpainting Funcs

In [243]:
def ddim_sample_inpainting_loop(
    self,
    model,
    shape,
    target_img,
    keep_mask,
    noise=None,
    fixed_noise=False,
    clip_denoised=True,
    denoised_fn=None,
    cond_fn=None,
    model_kwargs=None,
    device=None,
    progress=False,
    eta=0.0,
    debug=False,
):
    """
    Use DDIM to sample from the model and yield intermediate samples from
    each timestep of DDIM.
    Same usage as p_sample_loop_progressive().
    """
    if device is None:
        device = next(model.parameters()).device
    assert isinstance(shape, (tuple, list))
    if noise is not None:
        img = noise
    else:
        img = th.randn(*shape, device=device)
    indices = list(range(self.num_timesteps))[::-1]
    init_noise = img.clone()
    if progress:
        # Lazy import so that we don't depend on tqdm.
        from tqdm.auto import tqdm

        indices = tqdm(indices)

    for i in indices:
        t = th.tensor([i] * shape[0], device=device)
        with th.no_grad():
            
            out = self.ddim_sample(
                model,
                img,
                t,
                clip_denoised=clip_denoised,
                denoised_fn=denoised_fn,
                cond_fn=cond_fn,
                model_kwargs=model_kwargs,
                eta=eta,
            )
            img = out["sample"]
            
            alphas_cumprod = _extract_into_tensor(self.alphas_cumprod_prev, t, shape)
            alpha_t = th.sqrt(alphas_cumprod)
            sigma_t = th.sqrt(1 - alphas_cumprod)
            # inpaint the image by fixing the observed pixels
            if fixed_noise:
                target_img_noised = target_img * alpha_t + init_noise * sigma_t
            else:
                target_img_noised = target_img * alpha_t + th.randn_like(img) * sigma_t
            if debug:
                # print(t, alpha_t[0,].item(), sigma_t[0,].item())
                print(target_img_noised.shape, target_img.shape, keep_mask.shape, img.shape)
                print(target_img_noised[0].norm(), img[0].norm())
            img = img * keep_mask[None, None] + target_img_noised * (1 - keep_mask[None, None])
            
    return img



def _extract_into_tensor(arr, timesteps, broadcast_shape):
    """
    Extract values from a 1-D numpy array for a batch of indices.
    :param arr: the 1-D numpy array.
    :param timesteps: a tensor of indices into the array to extract.
    :param broadcast_shape: a larger shape of K dimensions with the batch
                            dimension equal to the length of timesteps.
    :return: a tensor of shape [batch_size, 1, ...] where the shape has K dims.
    """
    res = th.from_numpy(arr).to(device=timesteps.device)[timesteps].float()
    while len(res.shape) < len(broadcast_shape):
        res = res[..., None]
    return res + th.zeros(broadcast_shape, device=timesteps.device)

In [ ]:
@torch.no_grad()
def edm_sampler_inpaint(
    edm, latents, target_img, mask, class_labels=None,
    num_steps=18, sigma_min=0.002, sigma_max=80, rho=7,
    use_ema=True, fixed_noise=False
):
    # Adjust noise levels based on what's supported by the network.
    sigma_min = max(sigma_min, edm.sigma_min)
    sigma_max = min(sigma_max, edm.sigma_max)

    # Time step discretization.
    step_indices = torch.arange(num_steps, dtype=torch.float64, device=latents.device)
    t_steps = (sigma_max ** (1 / rho) + step_indices / (num_steps - 1) * (sigma_min ** (1 / rho) - sigma_max ** (1 / rho))) ** rho
    t_steps = torch.cat([edm.round_sigma(t_steps), torch.zeros_like(t_steps[:1])]) # t_N = 0
    initial_noise = torch.randn_like(latents)
    # Main sampling loop.
    x_next = latents.to(torch.float64) * t_steps[0]
    for i, (t_cur, t_next) in enumerate(zip(t_steps[:-1], t_steps[1:])): # 0, ..., N-1
        # x_hat = x_next
        t_hat = t_cur
        noise_perturb = initial_noise if fixed_noise else torch.randn_like(target_img)
        x_hat = (1 - mask[None, None]) * (target_img + noise_perturb * t_cur) + \
                     mask[None, None]  * x_next
        # Euler step.
        denoised = edm(x_hat, t_hat, class_labels, use_ema=use_ema).to(torch.float64)
        d_cur = (x_hat - denoised) / t_hat
        x_next = x_hat + (t_next - t_hat) * d_cur

        # Apply 2nd order correction.
        if i < num_steps - 1:
            denoised = edm(x_next, t_next, class_labels, use_ema=use_ema).to(torch.float64)
            d_prime = (x_next - denoised) / t_next
            x_next = x_hat + (t_next - t_hat) * (0.5 * d_cur + 0.5 * d_prime)

    return x_next

### Debug DDIM inpainting 

In [230]:
attr_all = np.load('/n/home12/binxuwang/Github/DiffusionReasoning/attr_all.npy')
# train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs_new.pt')
attr_all = torch.from_numpy(attr_all)
attr_all.shape

torch.Size([40, 12000, 3, 9, 3])

In [254]:
batch_size = 100
model_kwargs = dict(y=th.zeros(batch_size, dtype=torch.int, device="cuda"))
iclass = 30
idxs = [2,3,4]
# idxs = np.random.randint(0, 12000, 3)#np.random.choice(12000, 3)
#[3,4,5]
attr_tsr = train_data2attr_tsr(attr_all[iclass, idxs])
attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
print(r3_list, "row idxs", list(idxs), rule_col)
# assert np.all(orig_rules != -1)
assert len(r3_list[0]) == 1
orig_rule_id = r3_list[0][0]
target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
# for x_pos, y_pos in [(0, 2), (1, 2), (2, 2),]:
x_pos, y_pos = 2, 2
mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, fixed_noise=False,
                                            model_kwargs=model_kwargs, device="cuda", progress=False, eta=0.0)
samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
print(f"Rule [{orig_rule_id}] id[{idxs[x_pos]}] Consistent rules ratio inpainting samples ({x_pos}, {y_pos}): {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")

[[20]] row idxs [2, 3, 4] [[array([20]), array([20]), array([20])]]
Rule [20] id[4] Consistent rules ratio inpainting samples (2, 2): 9/100 (0.090)


In [256]:
(samples_inpaint[1] - attr_mtg.float()).round()

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  5.,  2.,  5.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0., -2.,  3.,  5.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  7.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  5.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0., -1.,  

In [257]:
(samples_inpaint[5] - attr_mtg.float()).round()

tensor([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  5.,  3.,  5.],
         [ 0.,  0.,  0.,  0.,  0.,  0., -2., -2.,  5.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  3.,  3.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  7.],
         [ 0.,  0.,  0.,  0.,  0.,  0., -7., -2.,  7.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  6.,  

In [258]:
samples_inpaint[5].round()

tensor([[[ 6.,  3.,  1., -1.,  0., -1., -1.,  6.,  1.],
         [-1.,  5., -1., -1., -1., -1.,  4.,  0., -1.],
         [ 0.,  2.,  1., -1., -1.,  5.,  5.,  3., -1.],
         [ 6., -1.,  6.,  1.,  4., -1.,  3.,  4., -1.],
         [-1., -1.,  0.,  6., -1., -1., -1., -1.,  1.],
         [ 0.,  0., -1.,  6., -1.,  4.,  0.,  1.,  0.],
         [ 4., -1.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
         [ 4.,  4.,  4.,  4.,  4.,  4., -1., -1.,  4.],
         [ 4.,  4.,  4., -1.,  4.,  4.,  4.,  4., -1.]],

        [[ 0.,  0.,  0., -1.,  1., -1., -1.,  3.,  3.],
         [-1.,  0., -1., -1., -1., -1.,  3.,  3., -1.],
         [ 0.,  0.,  0., -1., -1.,  1.,  3.,  3., -1.],
         [ 1., -1.,  1.,  3.,  3., -1.,  8.,  8., -1.],
         [-1., -1.,  1.,  3., -1., -1., -1., -1.,  8.],
         [ 1.,  1., -1.,  3., -1.,  3.,  8.,  8.,  8.],
         [ 2., -1.,  1.,  9.,  3.,  7.,  6.,  6.,  6.],
         [ 1.,  0.,  3.,  7.,  0.,  5., -1., -1.,  6.],
         [ 1.,  7.,  3., -1.,  5.,  2.,  6.,  

In [242]:
attr_mtg

tensor([[[ 4,  4,  4, -1,  4, -1, -1, -1,  1],
         [ 4,  4,  4,  4, -1, -1,  1, -1,  1],
         [ 4,  4,  4,  4,  4, -1,  1, -1,  1],
         [ 5,  5,  5, -1, -1, -1,  2, -1,  2],
         [ 5,  5,  5,  1,  1, -1,  2, -1, -1],
         [ 5,  5,  5, -1, -1,  1,  2,  2,  2],
         [-1,  5, -1, -1, -1, -1, -1,  2, -1],
         [ 5, -1,  5, -1,  4, -1, -1,  2, -1],
         [ 5, -1,  5, -1, -1,  4,  2, -1, -1]],

        [[ 7,  7,  7, -1,  7, -1, -1, -1,  9],
         [ 7,  7,  7,  7, -1, -1,  9, -1,  9],
         [ 7,  7,  7,  7,  7, -1,  9, -1,  9],
         [ 3,  3,  3, -1, -1, -1,  6, -1,  6],
         [ 3,  3,  3,  5,  5, -1,  6, -1, -1],
         [ 3,  3,  3, -1, -1,  5,  6,  6,  6],
         [-1,  4, -1, -1, -1, -1, -1,  9, -1],
         [ 4, -1,  4, -1,  3, -1, -1,  9, -1],
         [ 4, -1,  4, -1, -1,  3,  9, -1, -1]],

        [[ 4,  4,  4, -1,  3, -1, -1, -1,  3],
         [ 4,  4,  4,  3, -1, -1,  3, -1,  3],
         [ 4,  4,  4,  3,  3, -1,  3, -1,  3],
         

In [236]:
(samples_inpaint[0] - attr_mtg.float()).round()

tensor([[[ -0.,  -0.,  -0.,   0.,  -0.,   0.,   0.,   0.,   0.],
         [ -0.,  -0.,  -0.,  -0.,   0.,   0.,   0.,   0.,   0.],
         [ -0.,  -0.,  -0.,  -0.,  -0.,   0.,   0.,   0.,   0.],
         [ -1.,  -1.,  -1.,   0.,   0.,   0.,   0.,   0.,   0.],
         [ -1.,  -1.,  -1.,   0.,   0.,   0.,   0.,   0.,   0.],
         [ -1.,  -1.,  -1.,   0.,   0.,   0.,   0.,   0.,   0.],
         [  0.,  -1.,   0.,   0.,   0.,   0.,   5.,  -3.,   0.],
         [ -1.,   0.,  -1.,   0.,  -0.,   0.,   0.,  -3.,   5.],
         [ -1.,   0.,  -1.,   0.,   0.,  -0.,   2.,   0.,   0.]],

        [[ -1.,  -1.,  -1.,   0.,  -1.,   0.,   0.,   0.,  -3.],
         [ -1.,  -1.,  -1.,  -1.,   0.,   0.,  -3.,   0.,  -3.],
         [ -1.,  -1.,  -1.,  -1.,  -1.,   0.,  -3.,   0.,  -3.],
         [ -0.,  -0.,  -0.,   0.,   0.,   0.,   0.,   0.,   0.],
         [ -0.,  -0.,  -0.,  -0.,  -0.,   0.,   0.,   0.,   0.],
         [ -0.,  -0.,  -0.,   0.,   0.,  -0.,   0.,   0.,   0.],
         [  0.,  -0.,  

### In-paint

In [35]:
train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs_new.pt')
train_attrs.shape

torch.Size([35, 12000, 3, 9, 3])

In [177]:
iclass = 28
# iclass = 0
idxs = np.random.randint(0, 12000, 3)#np.random.choice(12000, 3)
#[3,4,5]
attr_tsr = train_data2attr_tsr(train_attrs[iclass, idxs])
attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
print(r3_list, rule_col)
# assert np.all(orig_rules != -1)
assert len(r3_list[0]) == 1
orig_rule_id = r3_list[0][0]

[[31]] [[array([31]), array([31]), array([31])]]


In [181]:
x_pos, y_pos = 2, 2
target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1

batch_size = 100
model_kwargs = dict(y=th.zeros(batch_size, dtype=torch.int, device="cuda"))

In [182]:
samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, 
                            denoised_fn=None, cond_fn=None, model_kwargs=model_kwargs, device="cuda", progress=True, eta=0.0)
samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
print(f"Rule [{orig_rule_id}] Consistent rules in inpainting samples ({x_pos}, {y_pos}): {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")

  0%|          | 0/200 [00:00<?, ?it/s]

Rule [31] Consistent rules in inpainting samples (2, 2): 81/100 (0.810)


### Massive production

In [ ]:
train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs_new.pt')
train_attrs.shape

In [185]:

attr_all = np.load('/n/home12/binxuwang/Github/DiffusionReasoning/attr_all.npy')
# train_attrs = torch.load('/n/home12/binxuwang/Github/DiffusionReasoning/train_inputs_new.pt')
attr_all = torch.from_numpy(attr_all)
attr_all.shape

(40, 12000, 3, 9, 3)

In [215]:
idx_seq

array([ 3478,   265,  2318, 11876, 10986,  7635,  2916,  5756, 10191,
        7269,  1914,    36, 10784,  5896,  9678])

In [262]:
def inpaint_run(iclass, idxs, batch_size=100):
    attr_tsr = train_data2attr_tsr(attr_all[iclass, idxs])
    attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
    r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
    print(r3_list, "row idxs", list(idxs), "rule", rule_col)
    # assert np.all(orig_rules != -1)
    assert len(r3_list[0]) == 1
    orig_rule_id = r3_list[0][0]
    target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
    for x_pos, y_pos in [(0, 2), (1, 2), (2, 2),]:
        mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
        mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
        samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, fixed_noise=False,
                                                    model_kwargs=model_kwargs, device="cuda", progress=False, eta=0.0)
        samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
        r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
        consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
        print(f"Rule [{orig_rule_id}] id[{idxs[x_pos]}] inpaint at ({x_pos}, {y_pos}) Consistent rules ratio : {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")

In [224]:
iclass = 36
inpaint_run(iclass, idx_seq[[1,2,5]])
inpaint_run(iclass, idx_seq[[2,5,1]])
inpaint_run(iclass, idx_seq[[5,1,2]])

[[36]] row idxs [265, 2318, 7635] rule [[array([36]), array([36]), array([36])]]
Rule [36] id[265] inpaint at (0, 2) Consistent rules ratio : 33/100 (0.330)
Rule [36] id[2318] inpaint at (1, 2) Consistent rules ratio : 52/100 (0.520)
Rule [36] id[7635] inpaint at (2, 2) Consistent rules ratio : 15/100 (0.150)
[[36]] row idxs [2318, 7635, 265] rule [[array([36]), array([36]), array([36])]]
Rule [36] id[2318] inpaint at (0, 2) Consistent rules ratio : 59/100 (0.590)
Rule [36] id[7635] inpaint at (1, 2) Consistent rules ratio : 13/100 (0.130)
Rule [36] id[265] inpaint at (2, 2) Consistent rules ratio : 37/100 (0.370)
[[36]] row idxs [7635, 265, 2318] rule [[array([36]), array([36]), array([36])]]
Rule [36] id[7635] inpaint at (0, 2) Consistent rules ratio : 19/100 (0.190)
Rule [36] id[265] inpaint at (1, 2) Consistent rules ratio : 32/100 (0.320)
Rule [36] id[2318] inpaint at (2, 2) Consistent rules ratio : 60/100 (0.600)


In [229]:
mask_tsr

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.],
        [0., 0., 0., 0., 0., 0., 1., 1., 1.]], device='cuda:0')

In [226]:
batch_size = 100
model_kwargs = dict(y=th.zeros(batch_size, dtype=torch.int, device="cuda"))
diffusion_eval = create_diffusion(timestep_respacing="ddim200")  # default: ddim100
iclass = 36
# iclass = 0
nreps = 5
idx_seq = np.random.choice(12000, 3 * nreps, replace=False)
for ireps in range(nreps):
    idxs = idx_seq[ireps*3:(ireps+1)*3]
    # idxs = np.random.randint(0, 12000, 3)#np.random.choice(12000, 3)
    #[3,4,5]
    attr_tsr = train_data2attr_tsr(attr_all[iclass, idxs])
    attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
    r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
    print(r3_list, "row idxs", list(idxs), rule_col)
    # assert np.all(orig_rules != -1)
    assert len(r3_list[0]) == 1
    orig_rule_id = r3_list[0][0]
    target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
    for x_pos, y_pos in [(0, 2), (1, 2), (2, 2),]:
        mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
        mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
        samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, fixed_noise=False,
                                                    model_kwargs=model_kwargs, device="cuda", progress=False, eta=0.0)
        samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
        r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
        consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
        print(f"Rule [{orig_rule_id}] id[{idxs[x_pos]}] Consistent rules ratio inpainting samples ({x_pos}, {y_pos}): {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")
        

[[36]] row idxs [4363, 487, 7306] [[array([36]), array([36]), array([36])]]
Rule [36] id[4363] Consistent rules ratio inpainting samples (0, 2): 43/100 (0.430)
Rule [36] id[487] Consistent rules ratio inpainting samples (1, 2): 38/100 (0.380)
Rule [36] id[7306] Consistent rules ratio inpainting samples (2, 2): 41/100 (0.410)
[[36]] row idxs [1760, 6236, 7850] [[array([36]), array([36]), array([36])]]
Rule [36] id[1760] Consistent rules ratio inpainting samples (0, 2): 16/100 (0.160)
Rule [36] id[6236] Consistent rules ratio inpainting samples (1, 2): 22/100 (0.220)
Rule [36] id[7850] Consistent rules ratio inpainting samples (2, 2): 55/100 (0.550)
[[36]] row idxs [2308, 1514, 900] [[array([36]), array([36]), array([36])]]
Rule [36] id[2308] Consistent rules ratio inpainting samples (0, 2): 11/100 (0.110)
Rule [36] id[1514] Consistent rules ratio inpainting samples (1, 2): 59/100 (0.590)
Rule [36] id[900] Consistent rules ratio inpainting samples (2, 2): 61/100 (0.610)
[[36]] row idxs [

## Massive Run

In [265]:
!echo $STORE_DIR

17016.04s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang


In [266]:
!ls /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects

17038.34s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


alexnet_rearing  edm_analy_traj_cmp    HaimDiffusionRNNProj
convrnn_Evol	 edm_score_validation  insilico_exps
DiT		 GPT-XL_grad_trace     mini_edm


In [267]:
savedir = "/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/DiT/results/045-RAVEN10_abstract-uncond-DiT_S_1_20240311-1256/inpaint_pilot"

In [278]:
import pandas as pd

batch_size = 100
model_kwargs = dict(y=th.zeros(batch_size, dtype=torch.int, device="cuda"))
diffusion_eval = create_diffusion(timestep_respacing="ddim200")  # default: ddim100
nreps = 50
sample_col = []
meta_col = []
rule_all_col = []
stats_col = []
# iclass = 0
for iclass in range(40):
    idx_seq = np.random.choice(12000, 3 * nreps, replace=False)
    for ireps in range(nreps):
        idxs = idx_seq[ireps*3:(ireps+1)*3]
        # idxs = np.random.randint(0, 12000, 3)#np.random.choice(12000, 3)
        attr_tsr = train_data2attr_tsr(attr_all[iclass, idxs])
        attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
        r3_list, r2_list, rule_col = infer_rule_from_sample_batch(attr_mtg)
        print(r3_list, "row idxs", list(idxs), rule_col)
        # assert np.all(orig_rules != -1)
        assert len(r3_list[0]) == 1
        orig_rule_id = r3_list[0][0]
        target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
        for x_pos, y_pos in [(0, 2), (1, 2), (2, 2),]:
            mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
            mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
            samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (batch_size, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, fixed_noise=False,
                                                        model_kwargs=model_kwargs, device="cuda", progress=False, eta=0.0)
            samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()
            r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
            consistent_r3 = sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])
            inpaint_rule_col = np.concatenate(tuple(np.array(rule_col_ipt, dtype=object)[:, x_pos]))
            if len(inpaint_rule_col):
                uniq, counts = np.unique(inpaint_rule_col, return_counts=True)
                # sort by counts
                idx_sort = np.argsort(counts)[::-1]
                uniq_sort = uniq[idx_sort]
                counts_sort = counts[idx_sort]
            else:
                uniq_sort = []
                counts_sort = []
            # print(f"Rule [{orig_rule_id}] id[{idxs[x_pos]}] Consistent rules ratio inpainting samples ({x_pos}, {y_pos}): {consistent_r3}/{batch_size} ({consistent_r3/batch_size:.3f})")
            meta_col.append({"iclass": iclass, "idxs": list(idxs), "rule": rule_col, "x_pos": x_pos, "y_pos": y_pos})
            sample_col.append(samples_inpaint)
            rule_all_col.append((r3_list_ipt, r2_list_ipt, rule_col_ipt))
            stats_col.append({"iclass": iclass, "idxs": list(idxs), "x_pos": x_pos, "y_pos": y_pos, "consistent_r3": consistent_r3, "batch_size": batch_size, 
                              "rule_uniq": uniq_sort, "rule_cnts": counts_sort}) # "rule": rule_col, 
        print(f"iclass {iclass} done {ireps}/{nreps}")
        df = pd.DataFrame(stats_col)
        df.to_csv(join(savedir, "inpaint_rule_stats.csv"))
                
    pkl.dump(sample_col, open(join(savedir, "inpaint_sample_col.pkl"), "wb"))
    pkl.dump(meta_col, open(join(savedir, "inpaint_meta_col.pkl"), "wb"))
    pkl.dump(rule_all_col, open(join(savedir, "inpaint_rule_all_col.pkl"), "wb"))
    pkl.dump(stats_col, open(join(savedir, "inpaint_stats_col.pkl"), "wb"))
df = pd.DataFrame(stats_col)
df.to_csv(join(savedir, "inpaint_rule_stats.csv"))


[[0]] row idxs [8513, 5917, 4432] [[array([0]), array([0]), array([0])]]
iclass 0 done 0/50
[[0]] row idxs [4114, 7145, 679] [[array([0]), array([0]), array([0])]]
iclass 0 done 1/50
[[0]] row idxs [59, 10568, 11170] [[array([0]), array([0]), array([0])]]
iclass 0 done 2/50
[[0]] row idxs [7776, 1633, 6888] [[array([0]), array([0]), array([0])]]
iclass 0 done 3/50
[[0]] row idxs [146, 597, 1108] [[array([0]), array([0]), array([0])]]
iclass 0 done 4/50
[[0]] row idxs [83, 9502, 2695] [[array([0]), array([0]), array([0])]]
iclass 0 done 5/50
[[0]] row idxs [9777, 2417, 2684] [[array([0]), array([0]), array([0])]]
iclass 0 done 6/50
[[0]] row idxs [2837, 7045, 7797] [[array([0]), array([0]), array([0])]]
iclass 0 done 7/50
[[0]] row idxs [10037, 10149, 10772] [[array([0]), array([0]), array([0])]]
iclass 0 done 8/50
[[0]] row idxs [7031, 8004, 10696] [[array([0]), array([0]), array([0])]]
iclass 0 done 9/50
[[0]] row idxs [3691, 10314, 6142] [[array([0]), array([0]), array([0])]]
iclass 

In [276]:
inpaint_rule_col = np.concatenate(tuple(np.array(rule_col_ipt, dtype=object)[:, x_pos]))
uniq, counts = np.unique(inpaint_rule_col, return_counts=True)
# sort by counts
idx_sort = np.argsort(counts)[::-1]
uniq_sort = uniq[idx_sort]
counts_sort = counts[idx_sort]
print(uniq_sort, counts_sort)

[25.  0. 30. 20. 26.  6.] [32 25 17  9  5  4]


### Scratch zone

In [227]:
(samples_inpaint[2] - attr_mtg).round()

tensor([[[ 0., -0., -0.,  0.,  0.,  0., -0.,  0., -0.],
         [ 0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.],
         [ 0., -0.,  0.,  0.,  0.,  0.,  0.,  0., -0.],
         [-0., -0.,  0., -0., -0., -0.,  0.,  0.,  0.],
         [-0., -0., -0.,  0.,  0., -0.,  0., -0.,  0.],
         [-0., -0., -0.,  0., -0., -0., -0.,  0.,  0.],
         [ 0., -0., -0.,  0.,  0.,  0., -4.,  1., -4.],
         [-0.,  0., -0.,  0.,  0., -0.,  0., -4., -4.],
         [-0., -0., -0.,  0.,  0., -0.,  1., -5.,  1.]],

        [[ 0., -2., -2.,  0.,  0.,  0., -1.,  0., -1.],
         [ 0., -2., -2.,  0.,  0.,  0.,  0., -1.,  0.],
         [ 0., -2.,  0.,  0.,  0.,  0.,  0.,  0., -1.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  2.,  7.,  2.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  2.],
         [ 0.,  0.,  0.,  0.,  0.,  0.,  7., -

In [207]:
rule_col_ipt

[[array([13]), array([13]), array([ 5, 13, 20])],
 [array([13]), array([13]), array([ 5, 30])],
 [array([13]), array([13]), array([13, 20])],
 [array([13]), array([13]), array([13])],
 [array([13]), array([13]), array([], dtype=float64)],
 [array([13]), array([13]), array([], dtype=float64)],
 [array([13]), array([13]), array([], dtype=float64)],
 [array([13]), array([13]), array([5])],
 [array([13]), array([13]), array([ 5, 13])],
 [array([13]), array([13]), array([20])],
 [array([13]), array([13]), array([], dtype=float64)],
 [array([13]), array([13]), array([13, 20])],
 [array([13]), array([13]), array([13])],
 [array([13]), array([13]), array([13, 20])],
 [array([13]), array([13]), array([5])],
 [array([13]), array([13]), array([13])],
 [array([13]), array([13]), array([ 5, 13, 20])],
 [array([13]), array([13]), array([13, 20])],
 [array([13]), array([13]), array([5])],
 [array([13]), array([13]), array([ 5, 20])],
 [array([13]), array([13]), array([ 5, 20])],
 [array([13]), array(

In [140]:
samples_inpaint = ddim_sample_inpainting_loop(diffusion_eval, model, (100, 3, 9, 9), target_img, mask_tsr, noise=None, clip_denoised=True, 
                            denoised_fn=None, cond_fn=None, model_kwargs=model_kwargs, device="cuda", progress=True, eta=0.0)
samples_inpaint = ((samples_inpaint.detach() * dataset_Xstd) + dataset_Xmean).cpu()

  0%|          | 0/100 [00:00<?, ?it/s]

torch.Size([100, 3, 9, 9]) torch.Size([1, 3, 9, 9]) torch.Size([9, 9]) torch.Size([100, 3, 9, 9])
tensor(16.3802, device='cuda:0') tensor(14.9425, device='cuda:0')
torch.Size([100, 3, 9, 9]) torch.Size([1, 3, 9, 9]) torch.Size([9, 9]) torch.Size([100, 3, 9, 9])
tensor(15.4465, device='cuda:0') tensor(16.3471, device='cuda:0')
torch.Size([100, 3, 9, 9]) torch.Size([1, 3, 9, 9]) torch.Size([9, 9]) torch.Size([100, 3, 9, 9])
tensor(15.6391, device='cuda:0') tensor(15.0314, device='cuda:0')
torch.Size([100, 3, 9, 9]) torch.Size([1, 3, 9, 9]) torch.Size([9, 9]) torch.Size([100, 3, 9, 9])
tensor(15.9289, device='cuda:0') tensor(15.9099, device='cuda:0')
torch.Size([100, 3, 9, 9]) torch.Size([1, 3, 9, 9]) torch.Size([9, 9]) torch.Size([100, 3, 9, 9])
tensor(15.7011, device='cuda:0') tensor(16.2316, device='cuda:0')
torch.Size([100, 3, 9, 9]) torch.Size([1, 3, 9, 9]) torch.Size([9, 9]) torch.Size([100, 3, 9, 9])
tensor(15.6725, device='cuda:0') tensor(15.7602, device='cuda:0')
torch.Size([100,

In [141]:
r3_list_ipt, r2_list_ipt, rule_col_ipt = infer_rule_from_sample_batch(samples_inpaint)
sum([orig_rule_id in cst_rule_set for cst_rule_set in r3_list_ipt])

61

In [142]:
rule_col_ipt

[[array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([ 6, 29])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([ 0, 29])],
 [array([0]), array([0]), array([6])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([6])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([ 6, 29])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([ 0, 19])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([29])],
 [array([0])

In [91]:
list(range(diffusion_eval.num_timesteps))[::-1]

[99,
 98,
 97,
 96,
 95,
 94,
 93,
 92,
 91,
 90,
 89,
 88,
 87,
 86,
 85,
 84,
 83,
 82,
 81,
 80,
 79,
 78,
 77,
 76,
 75,
 74,
 73,
 72,
 71,
 70,
 69,
 68,
 67,
 66,
 65,
 64,
 63,
 62,
 61,
 60,
 59,
 58,
 57,
 56,
 55,
 54,
 53,
 52,
 51,
 50,
 49,
 48,
 47,
 46,
 45,
 44,
 43,
 42,
 41,
 40,
 39,
 38,
 37,
 36,
 35,
 34,
 33,
 32,
 31,
 30,
 29,
 28,
 27,
 26,
 25,
 24,
 23,
 22,
 21,
 20,
 19,
 18,
 17,
 16,
 15,
 14,
 13,
 12,
 11,
 10,
 9,
 8,
 7,
 6,
 5,
 4,
 3,
 2,
 1,
 0]

In [83]:
diffusion_eval.alphas_cumprod_prev

array([1.00000000e+00, 9.99900000e-01, 9.97806573e-01, 9.93735429e-01,
       9.87710420e-01, 9.79766924e-01, 9.69951494e-01, 9.58321406e-01,
       9.44944106e-01, 9.29896552e-01, 9.13264481e-01, 8.95141591e-01,
       8.75628668e-01, 8.54832649e-01, 8.32865650e-01, 8.09843960e-01,
       7.85887024e-01, 7.61116416e-01, 7.35654824e-01, 7.09625052e-01,
       6.83149053e-01, 6.56347006e-01, 6.29336439e-01, 6.02231417e-01,
       5.75141786e-01, 5.48172503e-01, 5.21423031e-01, 4.94986820e-01,
       4.68950873e-01, 4.43395394e-01, 4.18393517e-01, 3.94011123e-01,
       3.70306733e-01, 3.47331484e-01, 3.25129172e-01, 3.03736366e-01,
       2.83182584e-01, 2.63490527e-01, 2.44676363e-01, 2.26750052e-01,
       2.09715711e-01, 1.93572010e-01, 1.78312584e-01, 1.63926474e-01,
       1.50398565e-01, 1.37710041e-01, 1.25838828e-01, 1.14760039e-01,
       1.04446407e-01, 9.48686970e-02, 8.59961101e-02, 7.77966584e-02,
       7.02375198e-02, 6.32853673e-02, 5.69066708e-02, 5.10679723e-02,
      

In [92]:
diffusion_eval.alphas_cumprod_prev

array([1.00000000e+00, 9.99900000e-01, 9.97806573e-01, 9.93735429e-01,
       9.87710420e-01, 9.79766924e-01, 9.69951494e-01, 9.58321406e-01,
       9.44944106e-01, 9.29896552e-01, 9.13264481e-01, 8.95141591e-01,
       8.75628668e-01, 8.54832649e-01, 8.32865650e-01, 8.09843960e-01,
       7.85887024e-01, 7.61116416e-01, 7.35654824e-01, 7.09625052e-01,
       6.83149053e-01, 6.56347006e-01, 6.29336439e-01, 6.02231417e-01,
       5.75141786e-01, 5.48172503e-01, 5.21423031e-01, 4.94986820e-01,
       4.68950873e-01, 4.43395394e-01, 4.18393517e-01, 3.94011123e-01,
       3.70306733e-01, 3.47331484e-01, 3.25129172e-01, 3.03736366e-01,
       2.83182584e-01, 2.63490527e-01, 2.44676363e-01, 2.26750052e-01,
       2.09715711e-01, 1.93572010e-01, 1.78312584e-01, 1.63926474e-01,
       1.50398565e-01, 1.37710041e-01, 1.25838828e-01, 1.14760039e-01,
       1.04446407e-01, 9.48686970e-02, 8.59961101e-02, 7.77966584e-02,
       7.02375198e-02, 6.32853673e-02, 5.69066708e-02, 5.10679723e-02,
      

In [88]:
diffusion_eval.alphas_cumprod_prev[diffusion_eval.timestep_map[-1]]

IndexError: index 990 is out of bounds for axis 0 with size 100

In [116]:
samples_inpaint[0].round().int()

tensor([[[-1,  4,  4, -1, -1,  4,  4, -1,  4],
         [ 4,  4, -1, -1, -1,  4,  4,  4, -1],
         [ 4, -1, -1, -1, -1, -1, -1, -1,  4],
         [ 5,  5,  5, -1, -1,  5, -1, -1, -1],
         [ 5,  5,  5,  5,  5, -1,  5, -1, -1],
         [ 5,  5,  5,  5, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1,  3, -1,  3],
         [-1, -1, -1, -1,  3, -1, -1,  3,  3],
         [-1, -1,  3,  3,  3,  3,  3,  3, -1]],

        [[-1,  5,  5, -1, -1,  1,  2, -1,  2],
         [ 5,  5, -1, -1, -1,  1,  2,  2, -1],
         [ 5, -1, -1, -1, -1, -1, -1, -1,  2],
         [ 6,  6,  6, -1, -1,  7, -1, -1, -1],
         [ 6,  6,  6,  7,  7, -1,  2, -1, -1],
         [ 6,  6,  6,  7, -1, -1, -1, -1, -1],
         [-1, -1, -1, -1, -1, -1,  1, -1,  1],
         [-1, -1, -1, -1,  6, -1, -1,  1,  1],
         [-1, -1,  8,  6,  6,  6,  1,  1, -1]],

        [[-1,  1,  1, -1, -1,  4,  3, -1,  3],
         [ 1,  1, -1, -1, -1,  4,  3,  3, -1],
         [ 1, -1, -1, -1, -1, -1, -1, -1,  3],
         

In [117]:
rule_col_ipt

[[array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([6])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([11, 24])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([25, 35])],
 [array([0]), array([0]), array([11])],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([], dtype=float64)],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([0])],
 [array([0]), array([0]), array([ 0, 11])],
 [array([0]), array([0]), array([ 0, 11])],
 [array([0]), array([0]), array([ 0, 35])],
 [array([0]), arra

In [78]:
total_steps = 40


iclass = 2
idxs = [3,4,5]
attr_tsr = train_data2attr_tsr(train_attrs[iclass, idxs])
attr_mtg = einops.rearrange(attr_tsr, "row attr h PW -> attr (row h) PW")
orig_rules = get_rule_img(attr_mtg)
assert len(np.unique(orig_rules)) == 1
assert np.all(orig_rules != -1)
orig_rule_id = np.unique(orig_rules)[0] # note this id can be different from iclass
x_pos, y_pos = 2, 2
target_img = (attr_mtg.float().to(device) - dataset_Xmean) / dataset_Xstd
mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1

In [36]:
device = "cuda"


In [ ]:
from diffusion import create_diffusion
diffusion_eval = create_diffusion(timestep_respacing="ddim100")  # default: ddim100

In [ ]:
idxs = [3,4,5]
attr_tsr = train_data2attr_tsr(train_attrs[iclass, idxs])
attr_mtg = einops.rearrange(attr_tsr, "b c h w -> c (b h) w")
orig_rules = get_rule_img(attr_mtg)
assert len(np.unique(orig_rules)) == 1
assert np.all(orig_rules != -1)
orig_rule_id = np.unique(orig_rules)[0] # note this id can be different from iclass
x_pos, y_pos = 1, 1
total_steps = 40
target_img = (attr_mtg.float().to(device) - Xmean) / Xstd
mask_tsr = torch.zeros(target_img.shape[-2:], device=device)
mask_tsr[3*x_pos:3*(x_pos+1), 3*y_pos:3*(y_pos+1)] = 1
latents = torch.randn(200, 3, 9, 9).to(device)
samples = edm_sampler_inpaint(edm, latents, target_img=target_img, mask=mask_tsr,
                                num_steps=total_steps, use_ema=False,
                                fixed_noise=True).detach()
# samples = edm_sampler(edm, latents, num_steps=num_steps)
samples = samples * Xstd + Xmean
samples_int = torch.round(samples).long().cpu()

rules_mat = []
for attr_mtg_inpaint in samples_int:
    rules = get_rule_img(attr_mtg_inpaint)
    rules_mat.append(rules)
rules_mat = np.asarray(rules_mat)
print(f"steps{num_steps} pos recovery rate {x_pos}, {y_pos} ", (rules_mat[:, x_pos]==orig_rule_id).mean(),
    "valid rate ", (rules_mat[:, x_pos]!=-1).mean())